In [8]:
import pandas as pd
import numpy as np
import uncertainties as uc

pd.options.display.float_format ='{:,.1e}'.format
import matplotlib.pyplot as plt
%matplotlib inline  
from IPython.display import Image
from IPython.core.display import HTML 

from openpyxl import load_workbook

from scipy import integrate
import scipy.stats as stats
from uncertainties import ufloat
from uncertainties import unumpy
from itertools import groupby

from scipy.stats import gmean

%run Utility_Functions.ipynb

# Result summary
This notebook able to run all other notebooks and gather all the results in one place

## Running all notebooks 

Creating a class that can load a notebook and execute it

In [9]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path



class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    exec(code, mod.__dict__);
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [11]:
%%capture  

#suppres the many outputs of the notebook for this run  
 
notebooks = ['Erythrocytes','Neutrophils','Lymphocytes','Monocytes','GIT Epithelial Cells', 'Skin cells',
             'Endothelial cells','Lung cells', 'Adipocytes','Myocytes','Cardiomyocytes','Brain Cells', 'Liver cells']
NBL = NotebookLoader()
for nb_name in notebooks:
    NBL.load_module(nb_name)


## Loading the concentated data
Importing from the excel into a CellTypesResDF object and using it to print all the results.
In addition, the object method enable to deal with the data. For example the extended Dataframe of all the results including the upper and lower bound is computed

In [12]:
cell_types_data = pd.read_excel('Summary.xlsx','Colors', index_col=0, usecols=range(4))
cell_types_data = cell_types_data.drop('Other')

cells = CellTypesResDF(cell_types_data.index,un_type='mixed')
cells.import_celltypes_from_xlsx('Summary.xlsx')
cells.print_params()
cells.get_extended_dataframe()

Erythrocytes:
Number of Erythrocytes is: (2.5±0.1)×10¹³ cells
Lifespan of Erythrocytes is: 119±3 days
Cellular turnover rate of Erythrocytes is: (2.1±0.1)×10¹¹ cells per day
Cell mass of Erythrocytes is: 94±3 pg
Cellular mass turnover rate of Erythrocytes is: 20.00±1.00 grams per day
Total cellular mass of Erythrocytes is: 2380±100 grams
______________________________
Neutrophils:
Number of Neutrophils is: (6.4±0.6)×10¹¹ cells
Lifespan of Neutrophils is: 6.6±0.2 days
Cellular turnover rate of Neutrophils is: (6.0±1.1)×10¹⁰ cells per day
Cell mass of Neutrophils is: 300±30 pg
Cellular mass turnover rate of Neutrophils is: 18±4 grams per day
Total cellular mass of Neutrophils is: 190±30 grams
______________________________
Monocytes:
Number of Monocytes is: 5e+09 cells (SD range: 4e+09 - 8e+09 cells)
Lifespan of Monocytes is: 4 days (SD range: 3 - 4 days)
Cellular turnover rate of Monocytes is: 2e+09 cells per day (SD range: 1e+09 - 2e+09 cells per day)
Cell mass of Monocytes is: 460 pg 

C:\Users\ronse\anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log10
C:\Users\ronse\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in remainder
C:\Users\ronse\anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less


,number,number lower bound,number upper bound,lifespan,lifespan lower bound,lifespan upper bound,cellular turnover rate,cellular turnover rate lower bound,cellular turnover rate upper bound,cell mass,...,total cellular mass upper bound,lifespan in rodents,lifespan in rodents lower bound,lifespan in rodents upper bound,extrapolated cellular turnover rate,extrapolated cellular turnover rate lower bound,extrapolated cellular turnover rate upper bound,extrapolated cellular mass turnover rate,extrapolated cellular mass turnover rate lower bound,extrapolated cellular mass turnover rate upper bound
cell type,,,,,,,,,,,,,,,,,,,,,
Erythrocytes,"25,500,000,000,000.0","25,000,000,000,000.0","26,000,000,000,000.0",119.0,116.0,122.0,"214,000,000,000.0","210,000,000,000.0","220,000,000,000.0",94.0,...,"2,500.0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Neutrophils,"640,000,000,000.0","580,000,000,000.0","700,000,000,000.0",6.6,6.4,6.8,"60,000,000,000.0","49,000,000,000.0","71,000,000,000.0",300.0,...,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Monocytes,"5,200,000,000.0","3,500,000,000.0","7,600,000,000.0",3.5,3.0,4.1,"1,520,000,000.0","1,000,000,000.0","2,300,000,000.0",460.0,...,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Thymocytes,"2,300,000,000.0","460,000,000.0","11,500,000,000.0",NaN,NaN,NaN,NaN,NaN,NaN,190.0,...,2.1,3.8,1.6,8.7,"610,000,000.0","100,000,000.0","3,700,000,000.0",0.1,0.0,0.7
Mature T cells,"710,000,000,000.0","320,000,000,000.0","1,560,000,000,000.0",320.0,240.0,430.0,"2,200,000,000.0","960,000,000.0","5,100,000,000.0",190.0,...,290.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B cells progenitors,"5,500,000,000.0","3,800,000,000.0","7,900,000,000.0",NaN,NaN,NaN,NaN,NaN,NaN,190.0,...,1.5,0.8,0.4,1.7,"6,600,000,000.0","3,700,000,000.0","11,900,000,000.0",1.2,0.7,2.2
Transitional B cells,"2,800,000,000.0","1,170,000,000.0","6,700,000,000.0",NaN,NaN,NaN,NaN,NaN,NaN,190.0,...,1.3,9.9,8.5,11.6,"290,000,000.0","121,000,000.0","700,000,000.0",0.1,0.0,0.1
Mature B cells,"320,000,000,000.0","145,000,000,000.0","700,000,000,000.0",63.0,52.0,76.0,"5,100,000,000.0","2,200,000,000.0","11,700,000,000.0",190.0,...,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dermal fibroblasts,"38,000,000,000.0","22,000,000,000.0","54,000,000,000.0",NaN,NaN,NaN,NaN,NaN,NaN,"2,300.0",...,130.0,140.0,120.0,160.0,"270,000,000.0","150,000,000.0","390,000,000.0",0.6,0.3,0.9
